<a href="https://colab.research.google.com/github/summermccune/Tokenization-Testing-for-Malware-Data/blob/main/Word2Vec/Word2Vec%26SVM_unigrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import glob
import nltk
from nltk.tokenize import word_tokenize
from sklearn.svm import SVC
from google.colab import drive
drive.mount('/content/drive')
import csv
from collections import Counter
import re
import gc

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def count(df, c):
  for row in df['Opcodes']:
     data = row.split()
     c.update(data)

In [ ]:
def removeNonVocab(vocab, series):
  rows = []
  vocab_str = '|'.join(vocab)
  pattern = '\\b((?!\\b( |' + vocab_str + ')\\b).)*\\b'
  for row in series:
    row = re.sub(pattern, '', row)
    row = re.sub(' +', ' ', row)
    rows.append(row)
  return rows

In [ ]:
from os import remove
#read in the data
FakeRean = pd.read_csv('/content/drive/My Drive/Data/FakeRean.csv')
OnLineGames = pd.read_csv('/content/drive/My Drive/Data/OnLineGames.csv')
Vobfus = pd.read_csv('/content/drive/My Drive/Data/Vobfus.csv')
Winwebsec = pd.read_csv('/content/drive/My Drive/Data/Winwebsec.csv')

FakeRean_df = pd.DataFrame(FakeRean)
OnLineGames_df = pd.DataFrame(OnLineGames)
Vobfus_df = pd.DataFrame(Vobfus)
Winwebsec_df = pd.DataFrame(Winwebsec)

#drop first column
FakeRean_df = FakeRean_df.iloc[:, 1:]
OnLineGames_df = OnLineGames_df.iloc[:, 1:]
Vobfus_df = Vobfus_df.iloc[:, 1:]

print(FakeRean_df.head())
print(FakeRean_df.shape)


      Label                                            Opcodes
0  FakeRean  add je testb je cmpb jb push push add add call...
1  FakeRean  push mov push push mov xor push mov cmp jne mo...
2  FakeRean  call mov pop pop pop ret int3 int3 int3 int3 i...
3  FakeRean  add in add push call pop ret int3 int3 int3 in...
4  FakeRean  mov push push push mov mov mov and cmp jne xor...
(1000, 2)


In [ ]:

#read in the data
FakeRean = pd.read_csv('/content/drive/My Drive/Data/FakeRean.csv')
OnLineGames = pd.read_csv('/content/drive/My Drive/Data/OnLineGames.csv')
Vobfus = pd.read_csv('/content/drive/My Drive/Data/Vobfus.csv')
Winwebsec = pd.read_csv('/content/drive/My Drive/Data/Winwebsec.csv')

FakeRean_df = pd.DataFrame(FakeRean)
OnLineGames_df = pd.DataFrame(OnLineGames)
Vobfus_df = pd.DataFrame(Vobfus)
Winwebsec_df = pd.DataFrame(Winwebsec)

#drop first column
FakeRean_df = FakeRean_df.iloc[:, 1:]
OnLineGames_df = OnLineGames_df.iloc[:, 1:]
Vobfus_df = Vobfus_df.iloc[:, 1:]
Winwebsec_df = Winwebsec_df.iloc[:, 1:]

#divide dfs in half
FakeRean_df = FakeRean_df.iloc[:len(FakeRean_df)//2]
OnLineGames_df = OnLineGames_df.iloc[:len(OnLineGames_df)//2]
Vobfus_df = Vobfus_df.iloc[:len(Vobfus_df)//2]
Winwebsec_df = Winwebsec_df.iloc[:len(Winwebsec_df)//2]

#splitting into test and train
# FakeRean_train, FakeRean_test = train_test_split(FakeRean, test_size=0.2, random_state = 7)
# OnLineGames_train, OnLineGames_test = train_test_split(OnLineGames, test_size=0.2, random_state = 7)
# Vobfus_train, Vobfus_test = train_test_split(Vobfus, test_size=0.2, random_state = 7)
# Winwebsec_train, Winwebsec_test = train_test_split(Winwebsec, test_size=0.2, random_state = 7)

#counting opcodes for data cleaning
unigramList = []
countTotal = Counter()
count(FakeRean_df, countTotal)
count(OnLineGames_df,countTotal)
count(Vobfus_df,countTotal)
count(Winwebsec_df,countTotal)

#list for most common opcodes
total_count = countTotal.most_common(31)
countList = [x[0] for x in total_count]
print(countList)

#adding cleaned data to new dataframe
rows = []

FakeRean_rows = removeNonVocab(countList, FakeRean_df['Opcodes'])
for row in FakeRean_rows:
  row = row.split()
  rows.append((tuple(row), 0))

OnLineGames_rows = removeNonVocab(countList, OnLineGames_df['Opcodes'])
for row in OnLineGames_rows:
  row = row.split()
  rows.append((tuple(row), 1))

Vobfus_rows = removeNonVocab(countList, Vobfus_df['Opcodes'])
for row in Vobfus_rows:
  row = row.split()
  rows.append((tuple(row), 2))

Winwebsec_rows = removeNonVocab(countList, Winwebsec_df['Opcodes'])
for row in Winwebsec_rows:
  row = row.split()
  rows.append((tuple(row), 3))


['add', 'mov', 'push', 'pop', 'inc', 'call', 'xchg', 'dec', 'or', 'cmp', 'xor', 'sub', 'lea', 'and', 'adc', 'sbb', 'test', 'out', 'in', 'jmp', 'movl', 'int3', 'ret', 'je', 'imul', 'nop', 'stos', 'lods', 'jne', 'scas', 'lret']


In [ ]:
#turn rows tuple into df
df = pd.DataFrame(rows, columns=['Opcodes', 'MalwareType'])
#remove rows from memory
del rows

In [ ]:
df.head()
#convert df to csv
df.to_csv('data.csv', index=False)

In [ ]:
#word2vec
model = Word2Vec(df['Opcodes'], min_count=1, vector_size=31, window=1)

In [ ]:
#SVM
X = np.array([np.mean([model.wv[word] for word in text if word in model.wv], axis=0) for text in df['Opcodes']])
y = df['MalwareType']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 7)

In [ ]:
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

#accuracy
print(svm.score(X_test, y_test))

#confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

0.78
[[73 10  2 25]
 [12 67  0  9]
 [ 0  6 96  0]
 [17  5  2 76]]
